<a href="https://colab.research.google.com/github/priyanshgupta1998/All_codes/blob/master/BigQuery_Data_Extract_on_google_colab%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://bigquery.cloud.google.com/dataset/hacker-news-241914:hacker_news?pli=1

https://console.cloud.google.com/iam-admin/settings?project=hacker-news-241914&folder=&organizationId=

https://googleapis.github.io/google-cloud-python/latest/bigquery/usage/client.html

# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Provide your credentials to the runtime

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


# Method1: ---> Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and displays the result, optionally saving it to a variable as a `DataFrame`.

In [3]:
%%bigquery --project hacker-news-241914 df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.hacker_news.stories`
#FROM `bigquery-public-data.samples.gsod`

,total_rows
0,1959809




---



---



---



---



---



---



---



---


# Method2: ----> Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://bigquery.cloud.google.com/table/bigquery-public-data:samples.gsod) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


In [8]:
project_id = 'hacker-news-241914'

#Sample approximately 2000 random rows
from google.cloud import bigquery

client = bigquery.Client(project='hacker-news-241914')

sample_count = 2000



row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.hacker_news.stories`''').to_dataframe().total[0]


df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.hacker_news.stories`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 1959809 rows


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1955 entries, 0 to 1954
Data columns (total 12 columns):
id             1955 non-null int64
by             1838 non-null object
score          1838 non-null float64
time           1936 non-null float64
time_ts        1936 non-null datetime64[ns, UTC]
title          1838 non-null object
url            1836 non-null object
text           1454 non-null object
deleted        98 non-null object
dead           395 non-null object
descendants    1743 non-null float64
author         1838 non-null object
dtypes: datetime64[ns, UTC](1), float64(3), int64(1), object(7)
memory usage: 183.4+ KB


In [48]:
df.head()

,id,by,score,time,time_ts,title,url,text,deleted,dead,descendants,author
0,2775680,None,NaN,1.310983e+09,2011-07-18 09:54:41+00:00,None,None,None,True,None,NaN,None
1,362337,None,NaN,1.226525e+09,2008-11-12 21:17:43+00:00,None,None,None,True,None,NaN,None
2,6720703,None,NaN,1.384288e+09,2013-11-12 20:24:39+00:00,None,None,None,True,None,NaN,None
3,842198,None,NaN,1.253826e+09,2009-09-24 21:06:45+00:00,None,None,None,True,None,NaN,None
4,2877053,None,NaN,1.313154e+09,2011-08-12 13:03:04+00:00,None,None,None,True,None,NaN,None


In [0]:
# 10 highest total_precipitation samples
#df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]





---



---



---


---



---



---



---



---


# Method3:--->Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [50]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=project_id, verbose=False, dialect='standard')

df.head()

/usr/local/lib/python3.6/dist-packages/pandas_gbq/gbq.py:810: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  "verbosity", FutureWarning, stacklevel=1)


,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028


In [0]:
# from google.cloud import bigquery
# client = bigquery.Client(project_id)